# Разработка классификатора новостей

**Нужно:**
* выбрать какой-либо новостной ресурс, где к новостям привязаны категории или метки (например http://lenta.ru, http://fontanka.ru, http://gazeta.ru)
* загрузить новости по некоторому набору (5-10) категорий за пару лет
* обучить классификатор на эти новостях
* продемонстрировать его работу, разработав простеший web-интерфейс (вариант - telegram-бот), куда пользователь вводит текст новости и на выходе получает наиболее вероятную категорию. В качестве фреймворка проще всего взять [Flask](http://flask.pocoo.org) (см. примеры) .

In [1]:
import pickle
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from tqdm import tqdm
from collections import Counter

In [2]:
categories = ['Технологии', 'ЖКХ', 'Происшествия', 'Власть', 'Общество', 'Строительство', 'Финансы',
              'Бизнес', 'Туризм', 'Авто', 'Спорт', 'Гиды']

## Парсинг

### Сформируем list со всеми линками на определенный день

In [12]:
urls = []
for year in [2016,2017]:
    for month in range(1,13):
        for day in range(1,29):
            if (day in range(1,10)) & (month in range(1,10)):
                urls.append('http://www.fontanka.ru/fontanka/' + str(year) + '/0' 
                            + str(month) + '/0' + str(day) + '/all.html')
            elif (day in range(1,10)) & (month in range(10,13)):
                urls.append('http://www.fontanka.ru/fontanka/' + str(year) + '/'
                                             + str(month) + '/0' + str(day) + '/all.html')
            elif (month in range(1,10)) & (day in range(10,29)):
                urls.append('http://www.fontanka.ru/fontanka/' + str(year) + '/0'
                            + str(month) + '/' + str(day) + '/all.html')
            else:
                urls.append('http://www.fontanka.ru/fontanka/' + str(year) + '/'
                            + str(month) + '/' + str(day) + '/all.html') 
def get_html(url):
    req = Request(url)
    webpage = urlopen(req)
    return webpage.read()

### Для каждого дня спарсим все линки на новости этого дня, также спарсим тэг новости и заголовок

In [13]:
tags = []
headers = []
links = []

for url in tqdm(urls):
    soup = BeautifulSoup(get_html(url), "html.parser")
    table = soup.find('div', attrs = {"class" : "calendar-list"})
    try:
        a = table.find_all('div', attrs = {"class" : "calendar-item-title"})
        b = table.find_all('div', attrs = {"calendar-item-category"})
    except:
        continue
    for i in range(len(a)):
        if (b[i].a.text in categories) & (len(a[i].a['href']) == 16):
            tags.append(b[i].a.text)
            headers.append(a[i].a.text)
            links.append(a[i].a['href'])
    #time.sleep(3)

100%|██████████| 672/672 [04:42<00:00,  2.38it/s]


### Спарсим все эти новости

In [ ]:
texts = []
errors = []
#for i in tqdm(range(50)):
for i in tqdm(range(len(links))):
    url = 'http://www.fontanka.ru' + links[i]
    try:
        soup = BeautifulSoup(get_html(url), "html.parser")
        table = soup.find('div', attrs = {"class" : "article_fulltext"})
        list_ = []
        string = ''
        for j in range(len(table.find_all('p'))):
            list_.append(table.find_all('p')[j].text.strip())
            string = ' '.join(list_)
        texts.append(''.join([headers[i], ' ', string]))
    except:
        errors.append(i)
        texts.append('')
        continue

100%|██████████| 61640/61640 [7:59:28<00:00,  2.14it/s]  


### Пока парсилось выключился wi-fi и некоторые новости не спарсились как надо, поэтому пройдемся заново по тем новостям, которые  вылетели с ошибкой

In [14]:
#texts = []
errors1 = []
#for i in tqdm(range(50)):
for i in tqdm(errors):
    url = 'http://www.fontanka.ru' + links[i]
    try:
        soup = BeautifulSoup(get_html(url), "html.parser")
        table = soup.find('div', attrs = {"class" : "article_fulltext"})
        list_ = []
        string = ''
        for j in range(len(table.find_all('p'))):
            list_.append(table.find_all('p')[j].text.strip())
            string = ' '.join(list_)
        texts[i] = ''.join([headers[i], ' ', string])
    except:
        errors1.append(i)
        texts[i] = ''
        continue

100%|██████████| 47/47 [00:04<00:00, 10.01it/s]


### Сохраним всё, что получили

In [47]:
with open('data.pickle', 'wb') as f:
    pickle.dump(texts, f)

In [17]:
with open('errors.pickle', 'wb') as f:
    pickle.dump(errors1, f)

In [49]:
with open('tags.pickle', 'wb') as f:
    pickle.dump(tags, f)

### Подготовка текста: токенизация -> лемматизация

In [2]:
from functools import lru_cache
import re
import pickle
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

@lru_cache(maxsize=100000)
def get_norm(word):
    return morph.parse(word)[0].normal_form

In [3]:
with open('errors.pickle', 'rb') as f:
    errors = pickle.load(f)
    
with open('data.pickle', 'rb') as f:
    texts = pickle.load(f)
    
with open('tags.pickle', 'rb') as f:
    tags = pickle.load(f)

In [4]:
print(len(tags))
print(len(texts))

61640
61640


In [5]:
Counter(tags)

Counter({'Авто': 1234,
         'Бизнес': 4624,
         'Власть': 6661,
         'Гиды': 3,
         'ЖКХ': 807,
         'Общество': 16121,
         'Происшествия': 22183,
         'Спорт': 5103,
         'Строительство': 1276,
         'Технологии': 764,
         'Туризм': 674,
         'Финансы': 2190})

### Удалим новости, которые ничего не содержат

In [6]:
texts = [e for e in texts if e not in '']
for i in range(len(errors)):
    if i > 0:
        del tags[errors[i] - i]
    else:
        del tags[errors[i]]

### Удалим новости с тэгом "Гиды", так как их всего 3

In [7]:
guides = np.where(np.array(tags) == 'Гиды')

In [8]:
for i in range(len(guides[0])):
    if i > 0:
        del tags[guides[0][i] - i]
        del texts[guides[0][i] - i]
    else:
        del tags[guides[0][i]]
        del texts[guides[0][i]]

### Избавимся от лишних новостей с тэгами "Происшествия" и "Общество"

In [9]:
prois = np.where(np.array(tags) == 'Происшествия')
for i in range(15000):
    if i > 0:
        del tags[prois[0][i] - i]
        del texts[prois[0][i] - i]
    else:
        del tags[prois[0][i]]
        del texts[prois[0][i]]

In [10]:
common = np.where(np.array(tags) == 'Общество')
for i in range(10000):
    if i > 0:
        del tags[common[0][i] - i]
        del texts[common[0][i] - i]
    else:
        del tags[common[0][i]]
        del texts[common[0][i]]

In [11]:
print(len(tags))
print(len(texts))

36622
36622


In [12]:
Counter(tags)

Counter({'Авто': 1234,
         'Бизнес': 4624,
         'Власть': 6659,
         'ЖКХ': 807,
         'Общество': 6118,
         'Происшествия': 7183,
         'Спорт': 5102,
         'Строительство': 1267,
         'Технологии': 764,
         'Туризм': 674,
         'Финансы': 2190})

### Токенизация

In [13]:
tokenized_texts = []
for i in range(len(texts)):
    words = re.findall('(?u)[А-Яа-яA-Za-z]+', texts[i])
    tokenized_texts.append(words)

### Лемматизация

In [14]:
for text in tqdm(range(len(tokenized_texts))):
    helpful_list = []
    for word in range(len(tokenized_texts[text])):
        helpful_list.append(get_norm(tokenized_texts[text][word]))
        if word == (len(tokenized_texts[text]) - 1):
            tokenized_texts[text] = helpful_list

100%|██████████| 36622/36622 [01:38<00:00, 372.49it/s]


In [15]:
X = []
for i in range(len(tokenized_texts)): 
    X.append(' '.join(tokenized_texts[i]))

### Обучение с помощью логистической регрессии, можно и SVM, либо naive bayes, качество почти одинаковое

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from collections import  Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

### Формируем tf-idf матрицу, также удаляем стоп слова

In [17]:
from stop_words import get_stop_words
stop_words = get_stop_words('russian')

vectorizer = TfidfVectorizer(min_df = 2, stop_words = stop_words)
matrix_X = vectorizer.fit_transform(X)

### Тут можно глянуть на качество

In [112]:
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(tags, n_folds = 5, shuffle = True)

In [19]:
# labels1 = np.array(tags)
# clf = LogisticRegression(C = 25, solver = 'newton-cg', multi_class = 'multinomial')
# clf =  LinearSVC()
# for train_index, test_index in skf:
#     train_index = np.array(train_index)
#     test_index = np.array(test_index)
#     X_train, X_test = matrix_X[train_index], matrix_X[test_index]
#     y_train, y_test = labels1[train_index], labels1[test_index]
#     clf.fit(X_train,y_train)
#     preds_train = clf.predict(X_test)
#     print(float(sum(preds_train == y_test)) / len(y_test))

### Обучаем

In [18]:
clf = LogisticRegression(C = 25, solver = 'newton-cg', multi_class = 'multinomial')
clf.fit(matrix_X, tags)

LogisticRegression(C=25, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

### Предсказывать будем на основе вектора размерности (1 х количество слов в tf-idf матрице), то есть будем формировать вектор такой размерности и будем добавлять вычисленный tf-idf для входного слова в столбец, соответствующий этому слова в tf-idf матрице.

In [29]:
word = 'доллар'
if word in vectorizer.vocabulary_:
    z = np.zeros((1,matrix_X.shape[1]))
    z[0, vectorizer.vocabulary_[word]] = 0.7
    print(clf.predict(z)[0])
else:
    print('Введите другое слово')

Финансы


Первая версия предсказателя. Работает только для одного слова. Фиксированное значение tf-idf

In [35]:
def check_word(word):
    if word in vectorizer.vocabulary_:
        z = np.zeros((1,matrix_X.shape[1]))
        z[0, vectorizer.vocabulary_[word]] = 0.7
        return clf.predict(z)[0]
    else:
        return 'Введите другое слово'

Вторая версия предсказателя. Работает для фраз. Считаем tf-idf для данного слова

In [70]:
def check_word(words):
    words = re.findall('(?u)[А-Яа-яA-Za-z]+', words)
    z = np.zeros((1,matrix_X.shape[1]))
    for word in words:
        norm_word = get_norm(word)
        if norm_word in vectorizer.vocabulary_:
            idf = np.log(36622/len(np.where(np.array(matrix_X[:,vectorizer.vocabulary_[norm_word]].todense()) != 0)[0]))
            tf = 1.0/len(words) #Будем считать, что слово встречается один раз в фразе
            z[0, vectorizer.vocabulary_[norm_word]] = tf * idf
                                                
    if np.sum(z) != 0:
        return clf.predict(z)[0]
    else:
        return 'Введите другое слово'

### Бот в 10 строк. Работать без включенного ноутбука, конечно, не будет.

In [ ]:
import telebot

token = ''
bot = telebot.TeleBot(token)

@bot.message_handler(content_types=["text"])


def messages(message):
    word = message.text
    #modified_word = get_norm(word)
    tag = check_word(word)#modified_word)
    bot.send_message(message.chat.id, tag)

if __name__ == '__main__':
    bot.polling(none_stop=True)

## Черновик